<a href="https://colab.research.google.com/github/berniewang8177/hello-world/blob/master/Libri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !wget https://www.openslr.org/resources/12/train-clean-100.tar.gz

In [ ]:
# !tar -xf train-clean-100.tar.gz

In [57]:
### Reference
import struct

def bytes_to_int(bytes: list) -> int:
        result = 0
        for byte in bytes:
            result = (result << 8) + byte
        return result


def get_flac_duration(filename: str) -> float:
    """
    Returns the duration of a FLAC file in seconds
    https://xiph.org/flac/format.html
    """
    with open(filename, 'rb') as f:
        if f.read(4) != b'fLaC':
            raise ValueError('File is not a flac file')
        header = f.read(4)
        while len(header):
            meta = struct.unpack('4B', header)  # 4 unsigned chars
            block_type = meta[0] & 0x7f  # 0111 1111
            size = bytes_to_int(header[1:4])

            if block_type == 0:  # Metadata Streaminfo
                streaminfo_header = f.read(size)
                unpacked = struct.unpack('2H3p3p8B16p', streaminfo_header)
                """
                https://xiph.org/flac/format.html#metadata_block_streaminfo
                16 (unsigned short)  | The minimum block size (in samples)
                                       used in the stream.
                16 (unsigned short)  | The maximum block size (in samples)
                                       used in the stream. (Minimum blocksize
                                       == maximum blocksize) implies a
                                       fixed-blocksize stream.
                24 (3 char[])        | The minimum frame size (in bytes) used
                                       in the stream. May be 0 to imply the
                                       value is not known.
                24 (3 char[])        | The maximum frame size (in bytes) used
                                       in the stream. May be 0 to imply the
                                       value is not known.
                20 (8 unsigned char) | Sample rate in Hz. Though 20 bits are
                                       available, the maximum sample rate is
                                       limited by the structure of frame
                                       headers to 655350Hz. Also, a value of 0
                                       is invalid.
                3  (^)               | (number of channels)-1. FLAC supports
                                       from 1 to 8 channels
                5  (^)               | (bits per sample)-1. FLAC supports from
                                       4 to 32 bits per sample. Currently the
                                       reference encoder and decoders only
                                       support up to 24 bits per sample.
                36 (^)               | Total samples in stream. 'Samples'
                                       means inter-channel sample, i.e. one
                                       second of 44.1Khz audio will have 44100
                                       samples regardless of the number of
                                       channels. A value of zero here means
                                       the number of total samples is unknown.
                128 (16 char[])      | MD5 signature of the unencoded audio
                                       data. This allows the decoder to
                                       determine if an error exists in the
                                       audio data even when the error does not
                                       result in an invalid bitstream.
                """

                samplerate = bytes_to_int(unpacked[4:7]) >> 4
                sample_bytes = [(unpacked[7] & 0x0F)] + list(unpacked[8:12])
                total_samples = bytes_to_int(sample_bytes)
                duration = float(total_samples) / samplerate

                return duration
            header = f.read(4)


### Create a CSV in the form of: 
audio_path, text content, length

In [4]:
import os
import pandas as pd
import numpy as np

In [21]:
first_level = "train-clean-100"
dataset_path = os.path.join( ".", "LibriSpeech", first_level)
sec_level = [ os.path.join(dataset_path, p) for p in os.listdir(dataset_path) ]

In [32]:
third_level = [ os.path.join(p,pp)  for p in sec_level for pp in os.listdir(p)]

In [68]:
def text_file(folder, paths):
  found = False
  content = []
  for path in paths:
    if "txt" in path:
      txt_path = os.path.join(folder, path)
      with open(txt_path) as f:
        lines = f.readlines()
        for line in lines:
          line = line.replace("\n", "").strip()
          split_line = line.split()
          sound_file = split_line[0] + ".flac"
          text = " ".join( split_line[1:] )
          sound_path = os.path.join(folder, sound_file)
          duration = get_flac_duration(sound_path)

          content.append([sound_path, text, duration])

      found = True
    if found:
      break
  return content
contents = None
for folder in third_level:
  # retrived text first as place holder
  paths = os.listdir(folder)
  content = text_file(folder, paths)
  if contents == None:
    contents = content
  else:
    contents += content
  # use the corresponding file title to retrieve the flac file
  # flac file duration

  pass

In [69]:
len(contents)

28539

In [75]:
contents[11100]

['./LibriSpeech/train-clean-100/7264/92310/7264-92310-0001.flac',
 'THE CAPITALIST OWNING THE SHEET WOULD NECESSARILY CONSIDER HIS REVENUE FROM ADVERTISEMENT BEFORE ANYTHING ELSE HE WAS INDEED COMPELLED TO DO SO UNLESS HE HAD ENORMOUS REVENUES FROM OTHER SOURCES AND RAN HIS PAPER AS A LUXURY COSTING A VAST FORTUNE A YEAR',
 13.975]

In [78]:
# df = pd.DataFrame(contents, columns = ['path', 'text', 'duration'])
# df.to_csv("train.csv", index = False)

In [89]:
# !wget https://www.openslr.org/resources/12/test-clean.tar.gz

--2022-09-22 12:16:21--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/test-clean.tar.gz [following]
--2022-09-22 12:16:21--  http://us.openslr.org/resources/12/test-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  28.9MB/s    in 12s     

2022-09-22 12:16:34 (26.7 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [90]:
# !tar -xf test-clean.tar.gz

In [96]:
# first_level = "test-clean"
# dataset_path = os.path.join( ".", "LibriSpeech", first_level)
# sec_level = [ os.path.join(dataset_path, p) for p in os.listdir(dataset_path) ]
# third_level = [ os.path.join(p,pp)  for p in sec_level for pp in os.listdir(p)]
# contents = None
# for folder in third_level:
#   # retrived text first as place holder
#   paths = os.listdir(folder)
#   content = text_file(folder, paths)
#   if contents == None:
#     contents = content
#   else:
#     contents += content
#   # use the corresponding file title to retrieve the flac file
#   # flac file duration

#   pass

In [97]:
# df = pd.DataFrame(contents, columns = ['path', 'text', 'duration'])
# df.to_csv("test.csv", index = False)

In [82]:
!git clone https://github.com/msalhab96/Conformer.git

Cloning into 'Conformer'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 138 (delta 71), reused 106 (delta 48), pack-reused 0
Receiving objects: 100% (138/138), 29.83 KiB | 4.97 MiB/s, done.
Resolving deltas: 100% (71/71), done.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Conformer  LibriSpeech	sample_data  train-clean-100.tar.gz  train.csv


In [ ]:
# https://www.openslr.org/resources/11/librispeech-vocab.txt

In [107]:
import torchaudio
torchaudio.__version__

'0.11.0+cu102'

In [106]:
path = "./LibriSpeech/train-clean-100/1116/137572/1116-137572-0000.flac"
x, sr = torchaudio.load(path, normalize = True)

In [100]:
contents

[['./LibriSpeech/test-clean/4077/13751/4077-13751-0000.flac',
  'ON THE SIXTH OF APRIL EIGHTEEN THIRTY THE CHURCH OF JESUS CHRIST OF LATTER DAY SAINTS WAS FORMALLY ORGANIZED AND THUS TOOK ON A LEGAL EXISTENCE',
  9.56],
 ['./LibriSpeech/test-clean/4077/13751/4077-13751-0001.flac',
  'ITS ORIGIN WAS SMALL A GERM AN INSIGNIFICANT SEED HARDLY TO BE THOUGHT OF AS LIKELY TO AROUSE OPPOSITION',
  8.745],
 ['./LibriSpeech/test-clean/4077/13751/4077-13751-0002.flac',
  'INSTEAD OF BUT SIX REGULARLY AFFILIATED MEMBERS AND AT MOST TWO SCORE OF ADHERENTS THE ORGANIZATION NUMBERS TODAY MANY HUNDRED THOUSAND SOULS',
  9.75],
 ['./LibriSpeech/test-clean/4077/13751/4077-13751-0003.flac',
  'IN PLACE OF A SINGLE HAMLET IN THE SMALLEST CORNER OF WHICH THE MEMBERS COULD HAVE CONGREGATED THERE NOW ARE ABOUT SEVENTY STAKES OF ZION AND ABOUT SEVEN HUNDRED ORGANIZED WARDS EACH WARD AND STAKE WITH ITS FULL COMPLEMENT OF OFFICERS AND PRIESTHOOD ORGANIZATIONS',
  16.72],
 ['./LibriSpeech/test-clean/4077/13751/